In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

public class ColorBoxGame
{
    private readonly Random _random = new Random();
    
    public char[,] GenerateGameMap(int rows, int cols, int colorVariants, bool includeSpaces = false, int spacePercentage = 10)
    {
        ValidateParameters(rows, cols, colorVariants, spacePercentage);

        var map = new char[rows, cols];
        var elements = PrepareGameElements(rows * cols, colorVariants, includeSpaces, spacePercentage);
        
        FillMap(map, elements);
        
        return map;
    }
    
    private void ValidateParameters(int rows, int cols, int colorVariants, int spacePercentage)
    {
        if (rows <= 0 || cols <= 0)
            throw new ArgumentException("Размеры карты должны быть положительными");
        
        if (colorVariants < 2 || colorVariants > 9)
            throw new ArgumentException("Количество цветов должно быть от 2 до 9");

        if (spacePercentage < 0 || spacePercentage > 50)
            throw new ArgumentException("Процент пробелов должен быть от 0 до 50");
    }
    
    private Queue<char> PrepareGameElements(int totalCells, int colorVariants, bool includeSpaces, int spacePercentage)
    {
        int spaceCount = includeSpaces ? (int)Math.Round(totalCells * spacePercentage / 100.0) : 0;
        int colorCells = totalCells - spaceCount;
        
        var colorCounts = CalculateColorDistribution(colorCells, colorVariants);
        var colors = Enumerable.Range('1', colorVariants).Select(c => (char)c).ToArray();
        
        var elements = new List<char>();
        
        // Добавляем цветные ячейки
        for (int i = 0; i < colorVariants; i++)
        {
            elements.AddRange(Enumerable.Repeat(colors[i], colorCounts[i]));
        }
        
        // Добавляем пробелы
        if (includeSpaces)
        {
            elements.AddRange(Enumerable.Repeat(' ', spaceCount));
        }
        
        // Перемешиваем
        return new Queue<char>(elements.OrderBy(_ => _random.Next()));
    }
    
    private int[] CalculateColorDistribution(int totalColorCells, int variants)
    {
        // Гарантируем разное количество для каждого цвета
        var counts = new List<int>();
        int remaining = totalColorCells;
        
        for (int i = 0; i < variants - 1; i++)
        {
            int max = remaining - (variants - i - 1);
            int count = _random.Next(1, max);
            counts.Add(count);
            remaining -= count;
        }
        
        counts.Add(remaining);
        
        // Перемешиваем распределение
        return counts.OrderBy(_ => _random.Next()).ToArray();
    }
    
    private void FillMap(char[,] map, Queue<char> elements)
    {
        for (int i = 0; i < map.GetLength(0); i++)
        {
            for (int j = 0; j < map.GetLength(1); j++)
            {
                map[i, j] = elements.Dequeue();
            }
        }
    }
    
    public void PrintGameMap(char[,] map, bool showGrid = true)
    {
        for (int i = 0; i < map.GetLength(0); i++)
        {
            for (int j = 0; j < map.GetLength(1); j++)
            {
                Console.Write(map[i, j]);
                if (showGrid) Console.Write(" ");
            }
            Console.WriteLine();
        }
    }
}

// Пример использования
var game = new ColorBoxGame();

Console.WriteLine("Стандартная карта 4x4:");
var basicMap = game.GenerateGameMap(4, 4, 3);
game.PrintGameMap(basicMap);

Console.WriteLine("\nКарта с пробелами 5x5:");
var spacedMap = game.GenerateGameMap(5, 5, 4, true, 25);
game.PrintGameMap(spacedMap);

Console.WriteLine("\nКомпактный вывод 3x6:");
var compactMap = game.GenerateGameMap(3, 6, 5, true, 15);
game.PrintGameMap(compactMap, false);

Стандартная карта 4x4:
3 3 2 3 
3 3 1 2 
2 3 3 2 
3 2 3 1 

Карта с пробелами 5x5:
2 4 4   2 
  2 4 1 4 
1   3 2 2 
4   3 4 1 
  1 2 4   

Компактный вывод 3x6:
4524 4
14144 
44 345
